# Data Loading

In [1]:
import pandas as pd
import numpy as np

In [2]:
df=pd.read_csv("Tweet_NFT.xlsx - Sheet1.csv",header=None)

# Data preprocessing

In [3]:
df=df.dropna()  

dropping NaN values for:
    1. To know the train size
    2. To do intial operations on training data(which is complete) 

In [4]:
df.shape

(96365, 4)

In [5]:
df.head()

,0,1,2,3
0,id,tweet_text,tweet_created_at,tweet_intent
1,1212762,@crypto_brody @eCoLoGy1990 @MoonrunnersNFT @It...,2022-08-06T16:56:36.000Z,Community
2,1212763,Need Sick Character artâ“#art #artist #Artist...,2022-08-06T16:56:36.000Z,Giveaway
3,1212765,@The_Hulk_NFT @INagotchiNFT @Tesla @killabears...,2022-08-06T16:56:35.000Z,Appreciation
4,1212766,@CryptoBatzNFT @DarekBTW The first project in ...,2022-08-06T16:56:35.000Z,Community


In [6]:
df=df.loc[1:,[1,3]]

In [7]:
df.head()

,1,3
1,@crypto_brody @eCoLoGy1990 @MoonrunnersNFT @It...,Community
2,Need Sick Character artâ“#art #artist #Artist...,Giveaway
3,@The_Hulk_NFT @INagotchiNFT @Tesla @killabears...,Appreciation
4,@CryptoBatzNFT @DarekBTW The first project in ...,Community
5,@sashadysonn The first project in crypto with ...,Community


In [8]:
df=df.reset_index(drop=True) #Resetting Indexes of the dataframe

In [9]:
df.describe()

,1,3
count,96364,96364
unique,84908,9
top,@Codex314159265 Feature it on @Globalnft07\nWe...,Community
freq,381,41510


# Data Cleaning

In [10]:
from nltk.tokenize import regexp_tokenize
from nltk.corpus import stopwords

In [11]:
main_words = stopwords.words("english")  #main words are some comman english words imported from stopards module

In [12]:
for i in range(df.shape[0]):
        df.loc[i,1]=regexp_tokenize(df.loc[i,1],'[a-zA-z]+')
        df.loc[i,1]=[i for i in df.loc[i,1] if i not in main_words]

Above loop makes data clean by removing numbers and any other speacial charactors and some comman words

In [13]:
lens=[len(i) for i in df.loc[:,1]]

In [14]:
max_length=max(lens) #gives me the maximum length of a list of main words of tweet

In [15]:
df.loc[0,1]

['crypto_brody',
 'eCoLoGy',
 'MoonrunnersNFT',
 'ItsukiNFT',
 'thelittlesnft',
 'SpaceRiders_NFT',
 'WDoopsies',
 'WomenandWeapons',
 'WoWPixiesNFT',
 'BokiNFT',
 'quirkiesnft',
 'bitcoin_bowie',
 'Kal_El_Krypto_',
 'sophiegreen',
 'A',
 'new',
 'cryptocurrency',
 'born',
 'fans',
 'Chihuahua',
 'meme',
 'community',
 'It',
 'simply',
 'pets',
 'love',
 'keep',
 'transaction',
 'automatically',
 'distributed',
 'CA',
 'x',
 'fd',
 'c',
 'B',
 'Dea',
 'E',
 'b',
 'f',
 'Chihuahua_bsc',
 'TG',
 'https',
 'co',
 'wqSzCivfaM']

# Splitting Data for training sets

In [16]:
xtrain=df.loc[:,1]

In [17]:
ytrain=df.loc[:,3]

In [18]:
ytrain

0           Community
1            Giveaway
2        Appreciation
3           Community
4           Community
             ...     
96359    Appreciation
96360         Presale
96361    Appreciation
96362       Community
96363    Appreciation
Name: 3, Length: 96364, dtype: object

# Final Data preperation for model Training using embeddings

In [19]:
f=open("glove.6B.50d.txt",encoding="UTF-8") # here I use pre Trained embeddings vector

In [20]:
embeddings123={}  #diction of embedded data words of around 6 billion words


for i in f:
    values=i.split()
    word=values[0]
    coeff=np.asarray(values[1:],dtype="float")
    #print(word,coeff)
    embeddings123[word]=coeff
    
f.close()

In [21]:
def embeddings_outputs(X):
    
    embeddings_out=np.zeros((X.shape[0],max_length,50))
    
    for i in range(X.shape[0]):
        # X.loc[i,1]=X.loc[i,1].split()
        
        for j in range(len(X[0])):
            try:
                embeddings_out[i][j]=embeddings123[X[i][j].lower()]
            except:
                 embeddings_out[i][j]=np.zeros((50,))
    
    return embeddings_out

In [22]:
embeddings_matrix_train=embeddings_outputs(xtrain)

Finally converted given data for model training (features class)

In [23]:
from sklearn import preprocessing
  
# label_encoder object knows how to understand word labels.
label_encoder = preprocessing.LabelEncoder()
  
# Encode labels in ytrain.
ytrain= label_encoder.fit_transform(ytrain)

In [24]:
from tensorflow.keras.utils import to_categorical

In [25]:
ytrain=to_categorical(ytrain,num_classes=9) #converted encodded data to as one hot vector


Finally converted given data for model training (Derived class)

# Model Building

In [26]:
from keras.models import Sequential
from keras.layers import *

In [27]:
model=Sequential()
model.add(LSTM(64,input_shape=(65,50),return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(64,return_sequences=False))
model.add(Dropout(0.5))
model.add(Dense(9))
model.add(Activation("softmax"))
model.compile(loss="categorical_crossentropy",optimizer="adam",metrics=["accuracy"])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 65, 64)            29440     
                                                                 
 dropout (Dropout)           (None, 65, 64)            0         
                                                                 
 lstm_1 (LSTM)               (None, 64)                33024     
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense (Dense)               (None, 9)                 585       
                                                                 
 activation (Activation)     (None, 9)                 0         
                                                                 
Total params: 63,049
Trainable params: 63,049
Non-traina

In [ ]:
hist=model.fit(embeddings_matrix_train,ytrain,epochs=10,batch_size=64,shuffle=True,validation_split=0.2) #model Training

Epoch 1/10
1205/1205 [==============================] - 94s 74ms/step - loss: 0.8586 - accuracy: 0.7109 - val_loss: 0.6050 - val_accuracy: 0.8057
Epoch 2/10
1205/1205 [==============================] - 91s 76ms/step - loss: 0.4950 - accuracy: 0.8461 - val_loss: 0.4933 - val_accuracy: 0.8402
Epoch 3/10
1205/1205 [==============================] - 88s 73ms/step - loss: 0.4108 - accuracy: 0.8729 - val_loss: 0.5028 - val_accuracy: 0.8396
Epoch 4/10
1205/1205 [==============================] - 78s 65ms/step - loss: 0.3709 - accuracy: 0.8837 - val_loss: 0.4544 - val_accuracy: 0.8499
Epoch 5/10
1205/1205 [==============================] - 98s 81ms/step - loss: 0.3484 - accuracy: 0.8917 - val_loss: 0.4198 - val_accuracy: 0.8653
Epoch 6/10
1205/1205 [==============================] - 80s 66ms/step - loss: 0.3177 - accuracy: 0.9012 - val_loss: 0.4035 - val_accuracy: 0.8713
Epoch 7/10
1205/1205 [==============================] - 82s 68ms/step - loss: 0.3061 - accuracy: 0.9050 - val_loss: 0.4388 -

# Data Loading, preprocessing and preparing for testing my model and prediction

PS: Following same steps as used above for training data

In [ ]:
df1=pd.read_csv("Tweet_NFT.xlsx - Sheet1.csv",header=None)

In [ ]:
df1=df1.loc[96365:,[1,3]]

In [ ]:
df1=df1.reset_index(drop=True)

In [ ]:
df1


In [ ]:
for i in range(df1.shape[0]):
        df1.loc[i,1]=regexp_tokenize(df1.loc[i,1],'[a-zA-z]+')
        df1.loc[i,1]=[i for i in df1.loc[i,1] if i not in main_words]

In [ ]:
lens=[len(i) for i in df1.loc[:,1]]

In [ ]:
max_length=max(lens)

In [ ]:
max_length

In [ ]:
df1.loc[0,1]

In [ ]:
xtest=df1.loc[:,1]

In [ ]:
embeddings_matrix_test=embeddings_outputs(xtest)

In [ ]:
embeddings_matrix_test

# Prediction Time

In [291]:
y=model.predict(embeddings_matrix_test)

In [292]:
y

array([[6.4441562e-02, 4.6273398e-01, 4.0621176e-02, ..., 1.8752927e-02,
        2.9430071e-01, 5.1447476e-04],
       [6.8940707e-03, 6.9994578e-04, 8.8187563e-04, ..., 1.8244582e-03,
        1.4167126e-02, 6.8765999e-05],
       [6.9867754e-03, 6.1286516e-03, 4.8420377e-04, ..., 8.9980889e-04,
        7.2342064e-03, 3.0298748e-05],
       ...,
       [6.9786496e-02, 2.3819473e-01, 3.7870184e-02, ..., 1.8532118e-02,
        5.1029325e-01, 5.0855271e-04],
       [4.7790521e-01, 8.8610120e-02, 5.9248652e-02, ..., 4.6350639e-02,
        1.5818998e-01, 5.9136312e-04],
       [8.5068196e-03, 7.5627756e-01, 1.3610853e-02, ..., 2.5372028e-03,
        1.3354321e-01, 1.6733908e-04]], dtype=float32)

### Cocnverting predicted array format back to required format

In [294]:
y=np.argmax(y,axis=1)

In [295]:
y= label_encoder.inverse_transform(y)

In [296]:
y

array(['Community', 'Giveaway', 'Giveaway', ..., 'Whitelist',
       'Appreciation', 'Community'], dtype=object)

ValueError: could not convert string to float: 'Community'

### Calling data again to add Predicted Array to it

In [332]:
df2=pd.read_csv("Tweet_NFT.xlsx - Sheet1.csv")

In [338]:
for i in range(len(df2.loc[96364:,"tweet_intent"])):
    df2.loc[96364+i:,"tweet_intent"]=y[i]
    

In [340]:
df2.loc[96362:,"tweet_intent"]

96362        Community
96363     Appreciation
96364        Community
96365         Giveaway
96366         Giveaway
              ...     
127448    Appreciation
127449       Community
127450       Whitelist
127451    Appreciation
127452       Community
Name: tweet_intent, Length: 31091, dtype: object

In [341]:
df2.head()

,id,tweet_text,tweet_created_at,tweet_intent
0,1212762,@crypto_brody @eCoLoGy1990 @MoonrunnersNFT @It...,2022-08-06T16:56:36.000Z,Community
1,1212763,Need Sick Character artâ“#art #artist #Artist...,2022-08-06T16:56:36.000Z,Giveaway
2,1212765,@The_Hulk_NFT @INagotchiNFT @Tesla @killabears...,2022-08-06T16:56:35.000Z,Appreciation
3,1212766,@CryptoBatzNFT @DarekBTW The first project in ...,2022-08-06T16:56:35.000Z,Community
4,1212767,@sashadysonn The first project in crypto with ...,2022-08-06T16:56:34.000Z,Community


# Result

In [342]:
df2 

,id,tweet_text,tweet_created_at,tweet_intent
0,1212762,@crypto_brody @eCoLoGy1990 @MoonrunnersNFT @It...,2022-08-06T16:56:36.000Z,Community
1,1212763,Need Sick Character artâ“#art #artist #Artist...,2022-08-06T16:56:36.000Z,Giveaway
2,1212765,@The_Hulk_NFT @INagotchiNFT @Tesla @killabears...,2022-08-06T16:56:35.000Z,Appreciation
3,1212766,@CryptoBatzNFT @DarekBTW The first project in ...,2022-08-06T16:56:35.000Z,Community
4,1212767,@sashadysonn The first project in crypto with ...,2022-08-06T16:56:34.000Z,Community
...,...,...,...,...
127448,1278721,"@Jason My company Universe is solving this, an...",2022-08-08T03:32:01.000Z,Appreciation
127449,1278723,@BosoTokyo Many will get rekt in this NFT,2022-08-08T03:32:00.000Z,Community
127450,1278724,@leeneedham81 @KibaInuWorld @HODL_OFFICIAL @Ko...,2022-08-08T03:32:00.000Z,Whitelist
127451,1278725,@1kz3ro @UNLEASHED_NFT I love this job,2022-08-08T03:31:59.000Z,Appreciation


## Finally Saving updated tweets with their intents

In [345]:
df2.to_csv("Tweet_NFT_Updated.csv",index=False) 